### Save the patient files for the Well-Being dataset in the format useful for the proposed framework. It also computes the serialised version of the data and the dynamic changes of the features.

In [21]:
import numpy as np
from scipy import stats
import pickle
from os import listdir
import os 
from os.path import isfile, join
from scipy.signal import savgol_filter

In [22]:
DATA_PATH = "full_fused_data"
DATA_PATH2 = "processed_data_smooth"
DATA_PATH3 = "all_video_audio"
DATA_PATH4 = "AllData"

onlyfiles = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]

In [23]:
partecipants_data = dict()
for i in range(len(onlyfiles)):
    tmp = onlyfiles[i].split("_")
    partecipants_ids = tmp[2].split(".")[0]
    for j in range(len(onlyfiles)):
        partecipants_files = list()
        tmp2 = onlyfiles[j].split("_")
        tmp3 = tmp2[2].split(".")[0]
        if (tmp3 == partecipants_ids):
            if (onlyfiles[j] not in partecipants_data.values()):
                partecipants_data.setdefault(partecipants_ids, [])
                partecipants_data[partecipants_ids].append(onlyfiles[j])
                
for k in partecipants_data.keys():
    partecipants_data[k] = set(partecipants_data[k])
        

In [24]:
speaker_label_data = {
    13: {
        0: [0]
    },
    14: {
        0: [1]
    },
    15: {
        0: [1]
    },
    18: {
        0: [2]
    },
    20: {
        0: [1]
    },
    29: {
        0: [0],
    },
    33: {
        0: [1]
    },
    39: {
        0: [1],
        1: [1],
        2: []  # Broken
    },
    45: {
        0: [1, 2],
        1: [0, 1],
        2: [1, 2, 3],
        3: [0, 1],
        4: [0, 1, 2, 3]
    },
    46: {
        0: [],  # Not Speaking
        1: [1]
    },
    64: {
        0:[1, 3],
    },
    25: {
        0:[1, 2],
        1: [1]
    },
    50: {
        0: [0, 2]
    },
    52: {
        0: [1],
        1: [0, 1]
    },
    56: {
        0: [1],
        1: [1]
    },
    58: {
        0: [1]
    },
    59: {
        0: [1],
        1: [1],
        2: [1],
        3: [2],
        4: [1, 3]
    },
    61: {
        0: [0]
    },
    65: {
        0: [0],
        1: [0, 1],
    },
    70: {
        0: [1],
        1: [1],
    },
    73: {
        0: [0, 2, 3],
        1: [0, 2, 3],
        2: [2],
        3: [1, 3],
        4: [3],
    },
    74: {
        0: [1, 2],
        1: [1, 2],
    },
    76: {
        0: [1],
        1: [0],
        2: [1],
    },
    86: {
        0: [2],
        1: [1],
    },
    87:{
        0: [1, 2, 3],
    },
    88:{
        0: [1],
    },
    89: {
        0: [1],
    },
    90: {
        0: [1],
        1: [0, 1, 3],
    },
    91: {
        0: [0, 2],
    },
    94: {
        0: [0, 1, 4],
    },
    100: {
        0: [0],
        1: [0],
        2: [0],
        3: [3],
    },
    102: {
        0: [1],
    },
    105: {
        0: [2],
        1: [1],
        2: [2],
    },
    112: {
        0: [1],
    }
}        

In [25]:
for participant_id in partecipants_data.keys():
    if participant_id in [109]:
        continue
    if int(participant_id) > 200:
        continue
    for session_id in partecipants_data[participant_id]:
        tmp = session_id.split("_")
        if (len(tmp)==4):
            ids = tmp[2]
            sess = tmp[3].split(".")[0]
        else:
            sess = 0
            ids = tmp[2].split(".")[0]
            
        
        fused_data_path = os.path.join(DATA_PATH, session_id)
        fused_data = pickle.load(open(fused_data_path, 'rb'))
        label = fused_data['label']
        fused_data = fused_data['data']
        #print(fused_data.shape)
        processed_data_path = os.path.join(DATA_PATH2, session_id)
        processed_data = np.load(processed_data_path)
        
        tmp = session_id.split(".")[0]
        tmp = tmp + ".wav_st.npy"
        mfcc_data_path = os.path.join(DATA_PATH3, tmp)
        mfcc_data = np.load(mfcc_data_path).T
        #print('mfcc data:', mfcc_data.shape)
        mfcc_data = mfcc_data[:fused_data.shape[0], 9:22] # MFCC data and data alignment
        #mfcc_data = stats.zscore(mfcc_data, axis=0, ddof=1)
        #print('final mfcc data', mfcc_data.shape)
    
        hand_cross_fidget_label_array = fused_data[:, 0].reshape((-1, 1))  # hand cross fidget data
        left_hand_arm_label_array = fused_data[:, 1].reshape((-1, 1))
        left_hand_leg_label_array = fused_data[:, 2].reshape((-1, 1))
        left_hand_face_label_array = fused_data[:, 3].reshape((-1, 1))
        right_hand_arm_label_array = fused_data[:, 4].reshape((-1, 1))
        right_hand_leg_label_array = fused_data[:, 5].reshape((-1, 1))
        right_hand_face_label_array = fused_data[:, 6].reshape((-1, 1))
        leg_location_label_array = fused_data[:, 7].reshape((-1, 1))
        leg_action_label_array = fused_data[:, 8].reshape((-1, 1))
        hand_action_label_array = fused_data[:, 9:11]  # shape (n, 2) NOTE!
        speaker_array = fused_data[:, 11].reshape((-1, 1)) # Speaker Identity
        voice_array = fused_data[:, 12].reshape((-1, 1)) # Slience (0) or not (1)
        
        left_hand_to_leg_fidget_array = np.zeros((fused_data.shape[0], 1))
        left_hand_to_leg_fidget_array[
            (hand_action_label_array[:, 0].reshape((-1, 1)) == 1) & (left_hand_leg_label_array == 1)] = 1

        right_hand_to_leg_fidget_array = np.zeros((fused_data.shape[0], 1))
        right_hand_to_leg_fidget_array[
            (hand_action_label_array[:, 1].reshape((-1, 1)) == 1) & (right_hand_leg_label_array == 1)] = 1

        left_hand_to_arm_fidget_array = np.zeros((fused_data.shape[0], 1))
        left_hand_to_arm_fidget_array[
            (hand_action_label_array[:, 0].reshape((-1, 1)) == 1) & (left_hand_arm_label_array == 1)] = 1

        right_hand_to_arm_fidget_array = np.zeros((fused_data.shape[0], 1))
        right_hand_to_arm_fidget_array[
            (hand_action_label_array[:, 1].reshape((-1, 1)) == 1) & (right_hand_arm_label_array == 1)] = 1

        left_hand_to_face_fidget_array = np.zeros((fused_data.shape[0], 1))
        left_hand_to_face_fidget_array[
            (hand_action_label_array[:, 0].reshape((-1, 1)) == 1) & (left_hand_face_label_array == 1)] = 1

        right_hand_to_face_fidget_array = np.zeros((fused_data.shape[0], 1))
        right_hand_to_face_fidget_array[
            (hand_action_label_array[:, 1].reshape((-1, 1)) == 1) & (right_hand_face_label_array == 1)] = 1

        leg_fidget_array = leg_action_label_array
        leg_fidget_array[leg_fidget_array > 1] = 1
        leg_fidget_array = np.array(savgol_filter(leg_fidget_array.reshape(-1).tolist(), 51, 3)).reshape(
            (-1, 1))
        leg_fidget_array[leg_fidget_array >= 0.8] = 1
        leg_fidget_array[leg_fidget_array < 0.8] = 0

        speaking_array = np.zeros((fused_data.shape[0], 1))
        for speaker_label in speaker_label_data[int(ids)][int(sess)]:
            speaking_array[(speaker_array == speaker_label)] = 1
        speaking_array[voice_array == 0] = 0
        
        # suppress signal when not speaking
        hand_cross_fidget_label_array[speaking_array == 0] = 0
        left_hand_to_leg_fidget_array[speaking_array == 0] = 0
        right_hand_to_leg_fidget_array[speaking_array == 0] = 0
        left_hand_to_arm_fidget_array[speaking_array == 0] = 0
        right_hand_to_arm_fidget_array[speaking_array == 0] = 0
        left_hand_to_face_fidget_array[speaking_array == 0] = 0
        right_hand_to_face_fidget_array[speaking_array == 0] = 0
        leg_fidget_array[speaking_array == 0] = 0
        
        #print(processed_data.shape)
        
        # Gaze data
        gaze_data = processed_data[:, list(range(274, 282))]
        gaze_data = stats.zscore(gaze_data, axis=0, ddof=1)
        
        # AU data
        AUs_data = processed_data[:, list(range(948, 983))]
        
        combined_data = np.hstack((
            hand_cross_fidget_label_array, #1
            left_hand_to_leg_fidget_array, #1
            right_hand_to_leg_fidget_array,#1
            left_hand_to_arm_fidget_array,#1
            right_hand_to_arm_fidget_array,#1
            left_hand_to_face_fidget_array,#1
            right_hand_to_face_fidget_array,#1
            leg_fidget_array,#1
            speaking_array,#1
            gaze_data,#8
            AUs_data,#35
            mfcc_data,#13
        ))
        
        print (combined_data.shape)
        label_column = [label] * combined_data.shape[0]
        combined_data = np.column_stack((combined_data, label_column))
        print (combined_data.shape)
        
        save_path = os.path.join(DATA_PATH4, session_id)
        print (save_path)
        np.save(save_path, combined_data)
        #fidget = combined_data[:,:9]
        #gaze = combined_data[:,9:17]
        #au = combined_data[:,17:52]
        #mfcc = combined_data[:,52:]
        
        

    

(14782, 65)
(14782, 66)
AllData/participant_video_56_1.npy
(1377, 65)
(1377, 66)
AllData/participant_video_56_0.npy
(20871, 65)
(20871, 66)
AllData/participant_video_102.npy
(10730, 65)
(10730, 66)
AllData/participant_video_76_1.npy
(7604, 65)
(7604, 66)
AllData/participant_video_76_2.npy
(3244, 65)
(3244, 66)
AllData/participant_video_76_0.npy
(22107, 65)
(22107, 66)
AllData/participant_video_112.npy
(21847, 65)
(21847, 66)
AllData/participant_video_52_0.npy
(3338, 65)
(3338, 66)
AllData/participant_video_52_1.npy
(22363, 65)
(22363, 66)
AllData/participant_video_74_0.npy
(5082, 65)
(5082, 66)
AllData/participant_video_74_1.npy
(30340, 65)
(30340, 66)
AllData/participant_video_50.npy
(20736, 65)
(20736, 66)
AllData/participant_video_91.npy
(554, 65)
(554, 66)
AllData/participant_video_46_0.npy
(20800, 65)
(20800, 66)
AllData/participant_video_46_1.npy
(2115, 65)
(2115, 66)
AllData/participant_video_86_1.npy
(8872, 65)
(8872, 66)
AllData/participant_video_86_0.npy
(4726, 65)
(4726, 66)

In [ ]:
DATA_PATH = "AllData"
DATA_PATH2 = "TemporalData60"
onlyfiles = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]

def get_dynamics(X_0th, time=0.1):
    """compute dynamics for data (1st)"""
    X_1st = np.zeros((X_0th.shape[0]-1, X_0th.shape[1]))
    for i in range(X_0th.shape[0]-1):
        X_1st[i] = (X_0th[i+1] - X_0th[i]) / time
    return X_1st

for i in range(len(onlyfiles)):
    
    tmp = np.load(os.path.join(DATA_PATH, onlyfiles[i]))
    label = tmp[:, -1]
    tmp = get_dynamics(tmp)
    tmp[:, -1] = label[1:]
    
    look_back = 60
    nb_samples = tmp.shape[0] - look_back
    x_train_reshaped = np.zeros((nb_samples, look_back, tmp.shape[1]))

    for k in range(nb_samples):
        y_position = k + look_back
        x_train_reshaped[k] = tmp[k:y_position]
    
        
    save_path = os.path.join(DATA_PATH2, onlyfiles[i])
    np.save(save_path, x_train_reshaped)
   